In [1]:
#Importing all the dependancies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy import stats
import statsmodels.api as sm
import math
import re
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sys
import pandas.core.algorithms as algos
pd.pandas.set_option('display.max_columns',None)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 3.9 MB/s 


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Loading Dataset
df = pd.read_csv('/content/drive/MyDrive/train.csv')
#test_data = pd.read_csv('E:\Fake News Detection/test.csv')
#submit = pd.read_csv('E:\Fake News Detection/submit.csv')
#df = pd.read_csv('train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


#**Data Preprocessing :**

In [7]:
df = df.fillna('')

In [8]:
#merging the train and test data across rows
#df = pd.concat([train_data, test_data], axis = 'rows')
#df.info()

In [9]:
#merging the "author" and "title" columns to form "Content"
df['content'] = df['author']+' '+df['title'] #+df['text']
df['content'].head()

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object

In [10]:
# separating the data & label
X = df.drop(columns='label', axis=1)
y = df['label']

In [11]:
#Defining a function "clean" to stem to the words in content
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
wordnet = WordNetLemmatizer
def clean(text):
    text="".join([re.sub('[^a-zA-Z]',' ',char) for char in text])
    text=text.lower()
    text=text.split()
    text=[stemmer.stem(word) for word in text if word not in set(stopwords.words("english"))]
    text=" ".join(text)
    return text

In [12]:
df['content'] = df['content'].apply(clean)

In [13]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [14]:
#Separating lables and values
X = df['content'].values
y = df['label'].values

In [15]:
print(X)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [16]:
y.shape

(20800,)

In [17]:
#Converting the textual data into numerical data using vertorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [34]:
#Splitting test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, stratify=y_train, random_state=42)

#**Implementing Logistic Regression :**

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [ ]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9850961538461539


In [ ]:
#Accuracy score on test data
X_test_pred = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, y_test)
print ('Accuracy score of the testing data :', test_data_accuracy)

Accuracy score of the testing data : 0.9706730769230769


In [ ]:
#Implementing Bagging ensemble on Logistic Regression Classifier
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(base_estimator = LogisticRegression(), n_estimators = 1000, max_features = 15000, random_state = 42)
bagging_clf.fit(X_train, y_train).score(X_test, y_test)

0.9670673076923076

#**Implementing XGBoost :**


In [19]:
pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [21]:
XGBClassifier().get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [22]:
model_xgboost = xgboost.XGBClassifier(n_estimators = 100,
                                     eval_metric = 'auc')
eval_set =[(X_val, y_val)]
model_xgboost.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=eval_set,
                  verbose=True)

[0]	validation_0-auc:0.928334
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.931941
[2]	validation_0-auc:0.940995
[3]	validation_0-auc:0.940963
[4]	validation_0-auc:0.940914
[5]	validation_0-auc:0.940995
[6]	validation_0-auc:0.940994
[7]	validation_0-auc:0.941017
[8]	validation_0-auc:0.941013
[9]	validation_0-auc:0.946845
[10]	validation_0-auc:0.946845
[11]	validation_0-auc:0.946851
[12]	validation_0-auc:0.949459
[13]	validation_0-auc:0.951292
[14]	validation_0-auc:0.951272
[15]	validation_0-auc:0.951272
[16]	validation_0-auc:0.95133
[17]	validation_0-auc:0.951283
[18]	validation_0-auc:0.953413
[19]	validation_0-auc:0.955607
[20]	validation_0-auc:0.957032
[21]	validation_0-auc:0.958454
[22]	validation_0-auc:0.958468
[23]	validation_0-auc:0.96179
[24]	validation_0-auc:0.963639
[25]	validation_0-auc:0.963661
[26]	validation_0-auc:0.963738
[27]	validation_0-auc:0.965232
[28]	validation_0-auc:0.96727
[29]	validation_0-auc:0.971439
[30]	validation_0-a

XGBClassifier(eval_metric='auc')

In [23]:
y_train_pred=model_xgboost.predict_proba(X_train)[:,1]
y_test_pred=model_xgboost.predict_proba(X_test)[:,1]
print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

y_train_pred=model_xgboost.predict(X_train)
y_test_pred=model_xgboost.predict(X_test)
print("Accuracy Train: {:.4f}\nAccuracy Test: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))

AUC Train: 0.9945
AUC Test: 0.9885
Accuracy Train: 0.9554
Accuracy Test: 0.9478


In [24]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model_xgboost, X, y, cv =15)

In [ ]:
score.max()

0.9704397981254506

In [ ]:
XGBClassifier().get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

#**Hyperparameter Tuning on XGBoost :**

In [27]:
params = {
    "learning_rate" : [0.01, 0.10, 0.25, 0.5, 1],
    "max_depth" : [3,5,10,15,20],
    "min_child_weight" : [1,3,5,7],
    "gamma" : [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree" : [0.3, 0.4, 0.5, 0.7, 1]
    
}

In [28]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [29]:
classifier = XGBClassifier()



###**Randomized Search CV :**


In [ ]:
#Hyperparameter tuning using Randomized Search CV
random_search =  RandomizedSearchCV(classifier, 
                                    param_distributions = params, n_iter=5, scoring ="roc_auc", n_jobs =-1, cv=5, verbose =3)

In [ ]:
random_search.fit(X,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5, estimator=XGBClassifier(), n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.7,
                                                             1],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01, 0.1, 0.25, 0.5,
                                                          1],
                                        'max_depth': [3, 5, 10, 15, 20],
                                        'min_child_weight': [1, 3, 5, 7]},
                   scoring='roc_auc', verbose=3)

In [ ]:
#Getting the best params from Random search
random_search.best_params_

{'colsample_bytree': 1,
 'gamma': 0.4,
 'learning_rate': 0.25,
 'max_depth': 15,
 'min_child_weight': 1}

In [ ]:
#Fitting the model with the best params we got
classifier = xgboost.XGBClassifier(min_child_weight= 1,
  max_depth= 5,
  learning_rate= 0.1,
  gamma= 0.1,
  colsample_bytree= 0.5,n_estimators = 100,eval_metric = 'auc')

In [ ]:
eval_set =[(X_val, y_val)]
classifier.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=eval_set,
                  verbose=True)

In [ ]:
y_train_pred=classifier.predict_proba(X_train)[:,1]
y_test_pred=classifier.predict_proba(X_test)[:,1]
print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

y_train_pred=classifier.predict(X_train)
y_test_pred=classifier.predict(X_test)
print("Accuracy Train: {:.4f}\nAccuracy Test: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))
                                            

AUC Train: 0.9955
AUC Test: 0.9901
Accuracy Train: 0.9718
Accuracy Test: 0.9675


In [ ]:
from sklearn.model_selection import cross_val_score
score_r = cross_val_score(classifier, X, y, cv =15)

In [ ]:
score_r

array([0.97692862, 0.97116078, 0.96755588, 0.96899784, 0.97476568,
       0.97837058, 0.97188176, 0.98702235, 0.97260274, 0.96827686,
       0.97402597, 0.96753247, 0.97258297, 0.97546898, 0.97041847])

In [ ]:
#Accuracy max
score_r.max()

0.9870223503965393

###**GridSearchCV :**

In [30]:
model_xgboost.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'eval_metric': 'auc',
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [31]:
classifier.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': None,
 'subsample': 1,
 'verbosity': 1}

In [32]:
params2 = {
    "learning_rate" : [0.1,0.5],
    "max_depth" : [5,9,11],
    "min_child_weight" : [1,3],
    "gamma" : [0.0,0.1,0.5,0.7],
    "colsample_bytree" : [0.5, 1]
    
}

In [35]:
grid_search = GridSearchCV( 
    estimator = classifier, param_grid = params2, scoring ="roc_auc", n_jobs =-1, cv=3, verbose =3)

grid_search = grid_search.fit(X_train,y_train)
#print(model.best_score_)
#print(model.best_estimator_.get_params())

Fitting 3 folds for each of 96 candidates, totalling 288 fits


In [ ]:
random_search.best_params_

{'colsample_bytree': 1,
 'gamma': 0.4,
 'learning_rate': 0.25,
 'max_depth': 15,
 'min_child_weight': 1}

In [36]:
#accuracy = gird_search.best_score_
grid_search.best_params_


{'colsample_bytree': 0.5,
 'gamma': 0.5,
 'learning_rate': 0.5,
 'max_depth': 11,
 'min_child_weight': 1}


best parameters so far -- {'colsample_bytree': 0.5,
 'gamma': 0.5,
 'learning_rate': 0.5,
 'max_depth': 11,
 'min_child_weight': 1}

In [43]:
#Fitting the model with the best params we got
classifier = xgboost.XGBClassifier(min_child_weight= 1,
  max_depth= 11,
  learning_rate= 0.5,
  gamma= 0.7,
  colsample_bytree= 0.5,n_estimators = 100,eval_metric = 'auc')

In [39]:
#eval_set =[(X_val, y_val)]
eval_set =[(X_test, y_test)]
classifier.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=eval_set,
                  verbose=True)

[0]	validation_0-auc:0.711296
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.957165
[2]	validation_0-auc:0.970727
[3]	validation_0-auc:0.979595
[4]	validation_0-auc:0.983892
[5]	validation_0-auc:0.988727
[6]	validation_0-auc:0.990234
[7]	validation_0-auc:0.990603
[8]	validation_0-auc:0.990844
[9]	validation_0-auc:0.991086
[10]	validation_0-auc:0.992512
[11]	validation_0-auc:0.993482
[12]	validation_0-auc:0.993597
[13]	validation_0-auc:0.99385
[14]	validation_0-auc:0.99457
[15]	validation_0-auc:0.994463
[16]	validation_0-auc:0.994769
[17]	validation_0-auc:0.994953
[18]	validation_0-auc:0.994996
[19]	validation_0-auc:0.995006
[20]	validation_0-auc:0.99546
[21]	validation_0-auc:0.995458
[22]	validation_0-auc:0.995543
[23]	validation_0-auc:0.995574
[24]	validation_0-auc:0.995627
[25]	validation_0-auc:0.995574
[26]	validation_0-auc:0.99561
[27]	validation_0-auc:0.995661
[28]	validation_0-auc:0.995566
[29]	validation_0-auc:0.995573
[30]	validation_0-au

XGBClassifier(colsample_bytree=0.5, eval_metric='auc', gamma=0.7,
              learning_rate=0.5, max_depth=11)

###**Results :**

In [40]:
#Accuracy with GridSearchCV hyperparameter tuning
y_train_pred=classifier.predict_proba(X_train)[:,1]
y_test_pred=classifier.predict_proba(X_test)[:,1]
print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))


y_train_pred=classifier.predict(X_train)
y_test_pred=classifier.predict(X_test)
print("Training accuracy after tuning the parameters using GridSearchCV: {:.4f}\nTest accuracy after tuning the parameters using GridSearchCV: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))
                                            

AUC Train: 0.9995
AUC Test: 0.9962
Training accuracy after tuning the parameters using GridSearchCV: 0.9946
Test accuracy after tuning the parameters using GridSearchCV: 0.9865


In [44]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)

score_g = cross_val_score(classifier, X, y, cv =skf, scoring = 'accuracy')

In [42]:
score_g.mean()

0.9921153846153846

###**Using Bayesian Optimization to tune parameteres on XGBoost :**

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, space_eval

In [ ]:

# Space
space = {
    'learning_rate': hp.choice('learning_rate', [ 0.01, 0.1, 0.5, 1]),
    'max_depth' : hp.choice('max_depth', [3,5,9,11]),
    'min_child_weight' : hp.choice('min_child_weight', range(1,4)),
    'gamma' : hp.choice('gamma', [i/10.0 for i in range(0,8)]),
    'colsample_bytree' : hp.choice('colsample_bytree', [i/10.0 for i in range(3,11)])
    #'reg_alpha' : hp.choice('reg_alpha', [0, 1e-5, 1e-2, 0.1, 1, 10, 100]), 
    #'reg_lambda' : hp.choice('reg_lambda', [1e-5, 1e-2, 0.1, 1, 10, 100])
}
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3)
# Objective function
def objective(params):
    
    xgboost = XGBClassifier(seed=0, **params)
    scores = cross_val_score(xgboost, X_train, y_train, cv=kfold, scoring='accuracy', n_jobs=-1)
    # Extract the best score
    best_score = max(scores)
    # Loss must be minimized
    loss = - best_score
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}
# Trials to track progress
bayes_trials = Trials()
# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = bayes_trials)

100%|██████████| 100/100 [19:19<00:00, 11.60s/it, best loss: -0.9877403846153846]


In [ ]:
# Print the index of the best parameters
print(best)
# Print the values of the best parameters
print(space_eval(space, best))

{'colsample_bytree': 0, 'gamma': 6, 'learning_rate': 2, 'max_depth': 3, 'min_child_weight': 0}
{'colsample_bytree': 0.3, 'gamma': 0.6, 'learning_rate': 0.5, 'max_depth': 11, 'min_child_weight': 1}


In [ ]:
#Fitting the model with the best params we got
classifier = xgboost.XGBClassifier(min_child_weight= 1,
  max_depth= 11,
  learning_rate= 0.5,
  gamma= 0.6,
  colsample_bytree= 0.3,
  n_estimators = 100,eval_metric = 'auc')

In [ ]:
eval_set =[(X_val, y_val)]
classifier.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=eval_set,
                  verbose=True)

[0]	validation_0-auc:0.711701
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.958172
[2]	validation_0-auc:0.967683
[3]	validation_0-auc:0.972946
[4]	validation_0-auc:0.986253
[5]	validation_0-auc:0.988337
[6]	validation_0-auc:0.990804
[7]	validation_0-auc:0.990301
[8]	validation_0-auc:0.991029
[9]	validation_0-auc:0.991407
[10]	validation_0-auc:0.992975
[11]	validation_0-auc:0.994797
[12]	validation_0-auc:0.995908
[13]	validation_0-auc:0.996062
[14]	validation_0-auc:0.996582
[15]	validation_0-auc:0.996645
[16]	validation_0-auc:0.99668
[17]	validation_0-auc:0.996749
[18]	validation_0-auc:0.996938
[19]	validation_0-auc:0.996962
[20]	validation_0-auc:0.997001
[21]	validation_0-auc:0.997172
[22]	validation_0-auc:0.997205
[23]	validation_0-auc:0.997223
[24]	validation_0-auc:0.997243
[25]	validation_0-auc:0.997273
[26]	validation_0-auc:0.997471
[27]	validation_0-auc:0.997552
[28]	validation_0-auc:0.997611
[29]	validation_0-auc:0.99753
[30]	validation_0-

XGBClassifier(colsample_bytree=0.3, eval_metric='auc', gamma=0.6,
              learning_rate=0.5, max_depth=11)

In [ ]:
y_train_pred=classifier.predict_proba(X_train)[:,1]
y_test_pred=classifier.predict_proba(X_test)[:,1]
print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))


y_train_pred=classifier.predict(X_train)
y_test_pred=classifier.predict(X_test)
print("Training accuracy after tuning the parameters using Bayesian Optimizer : {:.4f}\nTest accuracy after tuning the parameters using Bayesian Optimizer : {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))

AUC Train: 0.9997
AUC Test: 0.9961
Training accuracy after tuning the parameters using Bayesian Optimizer : 0.9954
Test accuracy after tuning the parameters using Bayesian Optimizer : 0.9856
